In [98]:
import requests
from bs4 import BeautifulSoup
from google.colab import files
import pandas as pd

In [99]:
#get USFM identifiers
URL = 'http://ubsicap.github.io/usfm/identification/books.html'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

stuff = [node.getText().split('\n')[1] for node in soup.findAll(class_=['row-odd', 'row-even'])]
book_codes = stuff[1:67]

In [100]:
#get list of links to book pages
URL = 'https://www.biblestudystart.com/outlines/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

links = []

for i, node in enumerate(soup.findAll('li')):

  link = node.find('a')

  if link is not None:
    fullLink = URL + link.get('href')
    links.append(fullLink)

#last two links are not relevant in this case
del links[-1]
del links[-1]

In [101]:
#create lists for all the fields we want in the end 
all_book_codes = []
all_chapters = []
all_start_verses = []
all_end_verses = []
all_summaries = []

#iterate through the link for each book
for i, link in enumerate(links): 

  new_page = requests.get(link)
  new_soup = BeautifulSoup(new_page.content, 'html.parser')

  #this string is formatted chapter:start_verse-end_verse
  chap_verse = [verse.getText().split() for verse in new_soup.findAll(class_ = "verses")]

  #need to fill in chapter info for Obadiah, where there is only one chapter and the website only lists verses
  for c_v in chap_verse:
    if len(c_v[1].split(':')) == 1:
      c_v[1] = "1:"+c_b[1]

  #get chapter info
  chapters = [c_v[1].split(':')[0] for c_v in chap_verse]
  verses = [c_v[1].split(':')[1] for c_v in chap_verse]

  #get start and end verses
  start_end = [v.split('-') for v in verses]

  #if there is only one verse listed, list it as both start and end
  for s_e in start_end:
    if len(s_e) == 1:
      s_e.append(s_e[0])

  #lists all info per book
  book_codes_i = [book_codes[i] for j in range(len(start_end))]
  start_verses = [s_e[0].split('.')[0] for s_e in start_end]
  end_verses = [s_e[1].split('.')[0] for s_e in start_end]
  summaries = [summary.getText() for summary in new_soup.findAll(class_ = "summary")]

  #append to the all lists
  for j in range(len(book_codes_i)):
    all_book_codes.append(book_codes_i[j])
    all_chapters.append(chapters[j])
    all_start_verses.append(start_verses[j])
    all_end_verses.append(end_verses[j])
    all_summaries.append(summaries[j])

In [102]:
#make df
data = {
    'Book':all_book_codes,
    'Chapter':all_chapters,
    'Start Verse':all_start_verses,
    'End Verse':all_end_verses,
    'Summary':all_summaries,
}

df = pd.DataFrame(data) 

In [103]:
#to csv
df.to_csv('pericopes.csv')